1) Se tiene un  RDD con el registro de notas de los alumnos de la forma (padron, materia, nota, fecha). Se pide resolver, utilizando PySpark:
**-A. Cuantos alumnos aprobaron al menos 1 materia en los ultimos 2 años;
-B. Un RDD conteniendo el promedoi de notas de cada alumno de la forma (padron, promedio);
-C. El nombre y apellido del alumno con mejor promedio. Para esto se puede usar un segundo RDD alumnos con registros (padron, nombre_y_apellido)**


Un RDD es un "Resilient Distributed Dataset" ->conjunto distribuido y flexible de datos

In [2]:
from fechasRandom import *
import random

registroDeNotas = []
padrones = []

fechaInicial = '2017-03-14'
fechaFinal = '2019-12-01'
for i in range(50):
    padron = random.randint(100900,101000)
    padrones.append(padron)
    materia = random.choice(["Fisica III","Laboratorio","Taller 1","Algebra II","Quimica"])
    nota = random.randint(1,10)
    fecha = random_date(fechaInicial, fechaFinal)
    registroDeNotas.append((padron,materia,nota,fecha))

In [3]:
notas = sc.parallelize(registroDeNotas)
notas.collect()

[(100946, 'Algebra II', 10, '2019-08-15'),
 (100940, 'Fisica III', 10, '2019-10-27'),
 (100965, 'Quimica', 8, '2017-08-31'),
 (100958, 'Taller 1', 4, '2018-12-28'),
 (100975, 'Fisica III', 3, '2017-08-11'),
 (100930, 'Taller 1', 4, '2017-04-24'),
 (100946, 'Laboratorio', 2, '2019-01-18'),
 (100991, 'Algebra II', 1, '2018-03-29'),
 (100966, 'Laboratorio', 6, '2018-08-20'),
 (100985, 'Quimica', 9, '2019-03-28'),
 (100918, 'Algebra II', 5, '2018-09-14'),
 (101000, 'Fisica III', 4, '2017-12-13'),
 (100937, 'Quimica', 7, '2019-07-15'),
 (100903, 'Laboratorio', 3, '2018-09-06'),
 (100941, 'Fisica III', 6, '2018-06-23'),
 (100962, 'Taller 1', 3, '2019-06-13'),
 (100938, 'Fisica III', 7, '2017-10-04'),
 (100986, 'Algebra II', 3, '2017-05-06'),
 (100902, 'Quimica', 3, '2017-08-15'),
 (100950, 'Taller 1', 7, '2017-08-20'),
 (100911, 'Laboratorio', 7, '2018-02-03'),
 (101000, 'Fisica III', 1, '2018-07-10'),
 (100968, 'Algebra II', 4, '2019-04-10'),
 (100951, 'Quimica', 3, '2018-11-04'),
 (100944,

**-A. Cuantos alumnos aprobaron al menos 1 materia en los ultimos 2 años**

In [4]:
#me quedo con las notas >= 4 y fechas de los ultimos 2 años
rddNotas = notas.filter(lambda x: (x[2]>=4) and (x[3]>'2017-03-14') and (x[3] <'2019-03-14'))
#me quedo con los padrones que aprobaron
rddNotas = rddNotas.map(lambda x: x[0])

#elimino los repetidos
rddNotas = rddNotas.distinct()
rddNotas.count()

21

**B. Un RDD conteniendo el promedoi de notas de cada alumno de la forma (padron, promedio)**

In [5]:
rddPromedios1 = notas.map(lambda x: (x[0],(x[2],1)))
#con reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) lo que hago es agrupar todos los valores con la misma clave (clave=padron, valor=(nota,1))
#x es (nota, 1), y es lasiguiente(nota,1), la salida es (nota+nota, 1+1)
rddPromedios = rddPromedios1.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))

#ahora mapeo por padron y por division entre suma de notas y cantidad de apariciones de ese padro
#map(lambda x: (x[0],x[1][0]/x[1][1])) donde x[0] es padron, x[1][0] es suma de notas y x[1][1] es cantidad de apariciones de ese padron
rddPromedios = rddPromedios.map(lambda x: (x[0], x[1][0]/x[1][1]))
rddPromedios.collect()

[(100946, 6.0),
 (100940, 6.0),
 (100965, 8.0),
 (100958, 4.0),
 (100975, 3.0),
 (100930, 6.5),
 (100991, 1.0),
 (100966, 3.5),
 (100985, 7.5),
 (100918, 5.0),
 (101000, 2.5),
 (100937, 7.5),
 (100903, 3.0),
 (100941, 6.0),
 (100962, 3.0),
 (100938, 7.0),
 (100986, 6.0),
 (100902, 3.0),
 (100950, 7.0),
 (100911, 7.0),
 (100968, 7.0),
 (100951, 4.5),
 (100944, 10.0),
 (100912, 10.0),
 (100993, 2.0),
 (100934, 5.5),
 (100977, 7.0),
 (100926, 9.0),
 (100919, 7.0),
 (100990, 8.0),
 (100907, 3.0),
 (100928, 4.5),
 (100956, 8.0),
 (100945, 5.0),
 (100981, 7.0),
 (100970, 9.0),
 (100910, 1.0),
 (100960, 1.0)]